# Digital Musicology (DH-401)
## Assignment 1: Discovering meter
Group 6
- Mickael Achkar
- Yichen Wang
- Yinghui Jiang

Rough outline
## 1. From musical score to metrical weights
## 2. From metrical weights to periodicities
## 3. From periodicities to time signature
## 4. Accuracy of classification

In [49]:
import ms3
import pandas as pd
import numpy as np
import math
import fractions
import matplotlib.pyplot as plt

In [80]:
meter_dataset = pd.read_csv('DM2022 - Assignment 1 [Dataset].csv', index_col = 0, converters = {'global_onset':fractions.Fraction, 'duration': fractions.Fraction})
useful_dataset = meter_dataset[["piece","timesig","global_onset", "staff", "voice", "duration", "nominal_duration", "tied", "gracenote"]]
# get minimum duration in noteset (ignore gracenotes)
no_grace_notes = useful_dataset[useful_dataset["gracenote"].isna()]
# no_grace_notes["duration"].min()

In [113]:
frame = no_grace_notes.groupby(["piece","staff"])["global_onset"].max()
total_durations = frame.to_dict()

for key,value in total_durations.items():
    total_durations[key] = float(value)

time_ranges = {}
for key,value in total_durations.items():
    time_ranges[key] = np.arange(0,value,1/128)

time_ranges
# we get 1/128. This minimum duration will be our sampling time.
# we need to sum up ties based on piece and staff and voice

# we will split all notes into 1/128, then sum up based on piece and staff and on global onset
# bin per song / staff
# merge notes with ties
# ignore grace notes or add them

{('K279-1',
  1): array([0.00000000e+00, 7.81250000e-03, 1.56250000e-02, ...,
        3.97976562e+02, 3.97984375e+02, 3.97992188e+02]),
 ('K279-1',
  2): array([0.00000000e+00, 7.81250000e-03, 1.56250000e-02, ...,
        3.97976562e+02, 3.97984375e+02, 3.97992188e+02]),
 ('K279-2',
  1): array([0.00000000e+00, 7.81250000e-03, 1.56250000e-02, ...,
        2.21226562e+02, 2.21234375e+02, 2.21242188e+02]),
 ('K279-2',
  2): array([0.00000000e+00, 7.81250000e-03, 1.56250000e-02, ...,
        2.21226562e+02, 2.21234375e+02, 2.21242188e+02]),
 ('K279-3',
  1): array([0.00000000e+00, 7.81250000e-03, 1.56250000e-02, ...,
        3.14476562e+02, 3.14484375e+02, 3.14492188e+02]),
 ('K279-3',
  2): array([0.00000000e+00, 7.81250000e-03, 1.56250000e-02, ...,
        3.14476562e+02, 3.14484375e+02, 3.14492188e+02]),
 ('K280-1',
  1): array([0.00000000e+00, 7.81250000e-03, 1.56250000e-02, ...,
        4.29976562e+02, 4.29984375e+02, 4.29992188e+02]),
 ('K280-1',
  2): array([0.00000000e+00, 7.81250